# RFM-Based Customer Segmentation Analysis

## Introduction & Methodology

### What is RFM Analysis?
RFM (Recency, Frequency, Monetary) analysis is a customer segmentation technique that uses past purchase behavior to divide customers into groups. It helps businesses identify their most valuable customers and tailor marketing strategies accordingly.

**RFM Components:**
- **Recency (R)**: How recently a customer made a purchase
- **Frequency (F)**: How often a customer makes purchases
- **Monetary (M)**: How much money a customer spends

### Customer Segments
We will segment customers into 6 categories:

| Segment | Description | Business Strategy |
|---------|-------------|-------------------|
| **Champion** | High spenders, visit often, long-term loyalty | Reward them, make them advocates |
| **Loyalist** | Consistent shoppers with strong engagement | Upsell higher value products |
| **Emerging** | Newer customers showing promising patterns | Nurture with targeted promotions |
| **Newbie** | Recent first-time shoppers | Onboard with welcome campaigns |
| **At Risk** | Visits have dropped off in recent weeks | Send re-engagement offers |
| **Hibernating** | Haven't shopped in a long time | Win-back campaigns with discounts |

## 1. Setup and Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("Libraries imported successfully!")

## 2. Data Loading & Preparation

In [ ]:
# Load the datasets
loyal_df = pd.read_csv('../data/loyal_customers_processed.csv')
new_df = pd.read_csv('../data/new_customers_processed.csv')

print(f"Loyal customers dataset: {loyal_df.shape[0]:,} rows, {loyal_df.shape[1]} columns")
print(f"New customers dataset: {new_df.shape[0]:,} rows, {new_df.shape[1]} columns")

In [ ]:
# Combine datasets
combined_df = pd.concat([loyal_df, new_df], ignore_index=True)

# Convert datetime column
combined_df['ticket_datetime'] = pd.to_datetime(combined_df['ticket_datetime'])
combined_df['date'] = pd.to_datetime(combined_df['date'])

print(f"\nCombined dataset: {combined_df.shape[0]:,} rows")
print(f"\nDate range: {combined_df['date'].min()} to {combined_df['date'].max()}")
print(f"\nUnique users: {combined_df['user_id'].nunique()}")
print(f"  - Loyal: {combined_df[combined_df['user_type'] == 'loyal']['user_id'].nunique()}")
print(f"  - New: {combined_df[combined_df['user_type'] == 'new']['user_id'].nunique()}")

In [ ]:
# Data validation
print("Data Validation:")
print(f"\nMissing values:")
print(combined_df[['user_id', 'ticket_number', 'ticket_datetime', 'ticket_amount']].isnull().sum())

print(f"\nSample data:")
combined_df.head()

## 3. RFM Metrics Calculation

### Calculating R, F, M for each customer:
- **Recency**: Days since customer's last purchase (from most recent date in dataset)
- **Frequency**: Number of unique transactions (ticket_numbers)
- **Monetary**: Total spending amount (sum of unique ticket_amounts)

In [ ]:
# Define the analysis date (most recent date in the dataset + 1 day)
analysis_date = combined_df['date'].max() + pd.Timedelta(days=1)
print(f"Analysis date: {analysis_date}")

# Create a user-level summary
# First, get unique ticket information (to avoid counting same ticket multiple times)
ticket_level = combined_df.groupby(['user_id', 'ticket_number', 'user_type']).agg({
    'ticket_datetime': 'first',
    'ticket_amount': 'first',
    'date': 'first'
}).reset_index()

print(f"\nUnique tickets: {ticket_level.shape[0]:,}")

In [ ]:
# Calculate RFM metrics per customer
rfm = ticket_level.groupby(['user_id', 'user_type']).agg({
    'date': lambda x: (analysis_date - x.max()).days,  # Recency
    'ticket_number': 'count',  # Frequency
    'ticket_amount': 'sum'  # Monetary
}).reset_index()

# Rename columns
rfm.columns = ['user_id', 'user_type', 'recency_days', 'frequency', 'monetary']

print(f"RFM table created for {rfm.shape[0]} customers")
rfm.head(10)

In [ ]:
# RFM Statistics
print("RFM Metrics Distribution:")
print("\n" + "="*50)
rfm[['recency_days', 'frequency', 'monetary']].describe()

In [ ]:
# Visualize RFM distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Recency distribution
axes[0].hist(rfm['recency_days'], bins=30, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Days Since Last Purchase')
axes[0].set_ylabel('Number of Customers')
axes[0].set_title('Recency Distribution')
axes[0].axvline(rfm['recency_days'].median(), color='red', linestyle='--', label=f'Median: {rfm["recency_days"].median():.0f}')
axes[0].legend()

# Frequency distribution
axes[1].hist(rfm['frequency'], bins=30, color='lightgreen', edgecolor='black')
axes[1].set_xlabel('Number of Transactions')
axes[1].set_ylabel('Number of Customers')
axes[1].set_title('Frequency Distribution')
axes[1].axvline(rfm['frequency'].median(), color='red', linestyle='--', label=f'Median: {rfm["frequency"].median():.0f}')
axes[1].legend()

# Monetary distribution
axes[2].hist(rfm['monetary'], bins=30, color='salmon', edgecolor='black')
axes[2].set_xlabel('Total Spending ($)')
axes[2].set_ylabel('Number of Customers')
axes[2].set_title('Monetary Distribution')
axes[2].axvline(rfm['monetary'].median(), color='red', linestyle='--', label=f'Median: ${rfm["monetary"].median():.2f}')
axes[2].legend()

plt.tight_layout()
plt.savefig('rfm_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: rfm_distributions.png")

## 4. RFM Scoring & Segmentation

### Scoring Methodology (1-5 scale using quintiles):
- **R Score**: 5 = most recent (lowest days), 1 = least recent
- **F Score**: 5 = highest frequency, 1 = lowest
- **M Score**: 5 = highest monetary, 1 = lowest

In [ ]:
# Create RFM scores using quintiles
# For Recency, lower is better, so we reverse the scoring
rfm['r_score'] = pd.qcut(rfm['recency_days'], q=5, labels=[5, 4, 3, 2, 1], duplicates='drop').astype(int)
rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5], duplicates='drop').astype(int)
rfm['m_score'] = pd.qcut(rfm['monetary'].rank(method='first'), q=5, labels=[1, 2, 3, 4, 5], duplicates='drop').astype(int)

# Create combined RFM score
rfm['rfm_score'] = rfm['r_score'].astype(str) + rfm['f_score'].astype(str) + rfm['m_score'].astype(str)

print("RFM Scores assigned!")
rfm.head(10)

In [ ]:
# Define segment assignment function
def assign_segment(row):
    r, f, m = row['r_score'], row['f_score'], row['m_score']
    
    # Champion: Recent + Frequent + High spender (R: 4-5, F: 4-5, M: 4-5)
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champion'
    
    # Loyalist: Frequent buyer, good engagement (R: 3-5, F: 4-5, M: 3-5)
    elif r >= 3 and f >= 4 and m >= 3:
        return 'Loyalist'
    
    # Newbie: Very recent, first-time buyer (R: 5, F: 1, M: 1-3)
    elif r == 5 and f == 1 and m <= 3:
        return 'Newbie'
    
    # Emerging: Recent, growing potential (R: 4-5, F: 2-3, M: 2-4)
    elif r >= 4 and f >= 2 and f <= 3 and m >= 2 and m <= 4:
        return 'Emerging'
    
    # At Risk: Was good, becoming inactive (R: 2-3, F: 3-5, M: 3-5)
    elif r >= 2 and r <= 3 and f >= 3 and m >= 3:
        return 'At Risk'
    
    # Hibernating: Long time since last visit (R: 1-2, F: 1-3, M: 1-5)
    elif r <= 2 and f <= 3:
        return 'Hibernating'
    
    # Default cases
    else:
        # Additional logic for edge cases
        if r >= 4:  # Recent but doesn't fit other categories
            return 'Emerging'
        elif r <= 2:  # Not recent
            return 'Hibernating'
        else:
            return 'At Risk'

# Apply segmentation
rfm['segment'] = rfm.apply(assign_segment, axis=1)

print("Segments assigned!")
print(f"\nSegment Distribution:")
print(rfm['segment'].value_counts())

In [ ]:
# Add segment descriptions
segment_descriptions = {
    'Champion': 'High spenders who visit often with long-term loyalty',
    'Loyalist': 'Consistent shoppers with strong engagement',
    'Emerging': 'Newer customers showing promising purchase patterns',
    'Newbie': 'Recent first-time shoppers with potential',
    'At Risk': 'Previously good customers whose visits have dropped off',
    'Hibernating': 'Inactive customers who have not shopped in a long time'
}

rfm['segment_description'] = rfm['segment'].map(segment_descriptions)
rfm.head(10)

## 5. Segment Analysis

In [ ]:
# Customer counts per segment
segment_counts = rfm['segment'].value_counts().reset_index()
segment_counts.columns = ['segment', 'customer_count']
segment_counts['percentage'] = (segment_counts['customer_count'] / segment_counts['customer_count'].sum() * 100).round(2)

print("Customer Distribution by Segment:")
print("="*50)
segment_counts

In [ ]:
# Revenue contribution by segment
segment_revenue = rfm.groupby('segment').agg({
    'monetary': ['sum', 'mean', 'median'],
    'frequency': ['sum', 'mean'],
    'recency_days': 'mean',
    'user_id': 'count'
}).round(2)

segment_revenue.columns = ['total_revenue', 'avg_revenue', 'median_revenue', 
                          'total_transactions', 'avg_transactions', 
                          'avg_recency_days', 'customer_count']
segment_revenue['revenue_share_%'] = (segment_revenue['total_revenue'] / segment_revenue['total_revenue'].sum() * 100).round(2)

# Define segment order for consistent display
segment_order = ['Champion', 'Loyalist', 'Emerging', 'Newbie', 'At Risk', 'Hibernating']
segment_revenue = segment_revenue.reindex(segment_order)

print("Segment Statistics:")
print("="*80)
segment_revenue

In [ ]:
# Loyal vs New customer breakdown per segment
user_type_segment = pd.crosstab(rfm['segment'], rfm['user_type'], margins=True)
user_type_segment = user_type_segment.reindex(segment_order + ['All'])

print("User Type Distribution by Segment:")
print("="*50)
user_type_segment

In [ ]:
# Average RFM scores by segment
rfm_by_segment = rfm.groupby('segment').agg({
    'r_score': 'mean',
    'f_score': 'mean',
    'm_score': 'mean',
    'recency_days': 'mean',
    'frequency': 'mean',
    'monetary': 'mean'
}).round(2)

rfm_by_segment = rfm_by_segment.reindex(segment_order)

print("Average RFM Metrics by Segment:")
print("="*70)
rfm_by_segment

## 6. Visualizations

In [ ]:
# Set up colors for segments
segment_colors = {
    'Champion': '#2ecc71',      # Green
    'Loyalist': '#3498db',      # Blue
    'Emerging': '#9b59b6',      # Purple
    'Newbie': '#f39c12',        # Orange
    'At Risk': '#e74c3c',       # Red
    'Hibernating': '#95a5a6'    # Gray
}

# Pie chart: Segment distribution
fig, ax = plt.subplots(figsize=(10, 8))

# Only include segments that have customers
existing_segments = [s for s in segment_order if s in rfm['segment'].values]
segment_data = rfm['segment'].value_counts().reindex(existing_segments)
colors = [segment_colors[seg] for seg in segment_data.index]

wedges, texts, autotexts = ax.pie(
    segment_data.values, 
    labels=segment_data.index,
    colors=colors,
    autopct='%1.1f%%',
    startangle=90,
    explode=[0.02] * len(segment_data)
)

ax.set_title('Customer Segment Distribution', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('segment_pie_chart.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: segment_pie_chart.png")

In [ ]:
# Bar charts: Segment comparisons
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Only include segments that have customers
existing_segments = [s for s in segment_order if s in rfm['segment'].values]

# Customer Count by Segment
segment_data = rfm['segment'].value_counts().reindex(existing_segments)
colors = [segment_colors[seg] for seg in segment_data.index]
axes[0, 0].bar(segment_data.index, segment_data.values, color=colors, edgecolor='black')
axes[0, 0].set_title('Customer Count by Segment', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Number of Customers')
axes[0, 0].tick_params(axis='x', rotation=45)
for i, v in enumerate(segment_data.values):
    axes[0, 0].text(i, v + 0.5, str(v), ha='center', fontweight='bold')

# Average Monetary Value by Segment
monetary_data = rfm.groupby('segment')['monetary'].mean().reindex(existing_segments)
colors = [segment_colors[seg] for seg in monetary_data.index]
axes[0, 1].bar(monetary_data.index, monetary_data.values, color=colors, edgecolor='black')
axes[0, 1].set_title('Average Spending by Segment', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Average Monetary Value ($)')
axes[0, 1].tick_params(axis='x', rotation=45)
for i, v in enumerate(monetary_data.values):
    axes[0, 1].text(i, v + 10, f'${v:.0f}', ha='center', fontweight='bold')

# Average Frequency by Segment
freq_data = rfm.groupby('segment')['frequency'].mean().reindex(existing_segments)
colors = [segment_colors[seg] for seg in freq_data.index]
axes[1, 0].bar(freq_data.index, freq_data.values, color=colors, edgecolor='black')
axes[1, 0].set_title('Average Transaction Frequency by Segment', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Average Transactions')
axes[1, 0].tick_params(axis='x', rotation=45)
for i, v in enumerate(freq_data.values):
    axes[1, 0].text(i, v + 0.5, f'{v:.1f}', ha='center', fontweight='bold')

# Total Revenue by Segment
revenue_data = rfm.groupby('segment')['monetary'].sum().reindex(existing_segments)
colors = [segment_colors[seg] for seg in revenue_data.index]
axes[1, 1].bar(revenue_data.index, revenue_data.values, color=colors, edgecolor='black')
axes[1, 1].set_title('Total Revenue Contribution by Segment', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Total Revenue ($)')
axes[1, 1].tick_params(axis='x', rotation=45)
for i, v in enumerate(revenue_data.values):
    axes[1, 1].text(i, v + 100, f'${v:,.0f}', ha='center', fontweight='bold', fontsize=8)

plt.tight_layout()
plt.savefig('segment_bar_charts.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: segment_bar_charts.png")

In [ ]:
# Scatter plot: Frequency vs Monetary by Segment
fig, ax = plt.subplots(figsize=(12, 8))

# Only include segments that have customers
existing_segments = [s for s in segment_order if s in rfm['segment'].values]

for segment in existing_segments:
    segment_df = rfm[rfm['segment'] == segment]
    ax.scatter(
        segment_df['frequency'], 
        segment_df['monetary'],
        c=segment_colors[segment],
        label=segment,
        alpha=0.7,
        s=100,
        edgecolor='white'
    )

ax.set_xlabel('Frequency (Number of Transactions)', fontsize=12)
ax.set_ylabel('Monetary Value ($)', fontsize=12)
ax.set_title('Customer Segments: Frequency vs Monetary Value', fontsize=14, fontweight='bold')
ax.legend(title='Segment', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('frequency_vs_monetary_scatter.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: frequency_vs_monetary_scatter.png")

In [ ]:
# Heatmap: RFM Score patterns by segment
# Only include segments that have customers
existing_segments = [s for s in segment_order if s in rfm['segment'].values]
rfm_scores_by_segment = rfm.groupby('segment')[['r_score', 'f_score', 'm_score']].mean().reindex(existing_segments)

fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(
    rfm_scores_by_segment, 
    annot=True, 
    cmap='RdYlGn',
    fmt='.2f',
    linewidths=0.5,
    ax=ax,
    vmin=1,
    vmax=5
)
ax.set_title('Average RFM Scores by Segment', fontsize=14, fontweight='bold')
ax.set_xlabel('RFM Component')
ax.set_ylabel('Segment')

plt.tight_layout()
plt.savefig('rfm_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: rfm_heatmap.png")

In [ ]:
# Stacked bar: Loyal vs New customers per segment
# Only include segments that have customers
existing_segments = [s for s in segment_order if s in rfm['segment'].values]
user_type_data = rfm.groupby(['segment', 'user_type']).size().unstack(fill_value=0).reindex(existing_segments)

fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(existing_segments))
width = 0.6

loyal_vals = user_type_data.get('loyal', pd.Series([0]*len(existing_segments), index=existing_segments))
new_vals = user_type_data.get('new', pd.Series([0]*len(existing_segments), index=existing_segments))

ax.bar(x, loyal_vals, width, label='Loyal', color='#3498db', edgecolor='black')
ax.bar(x, new_vals, width, bottom=loyal_vals, label='New', color='#f39c12', edgecolor='black')

ax.set_xlabel('Segment')
ax.set_ylabel('Number of Customers')
ax.set_title('Customer Type Distribution by Segment', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(existing_segments, rotation=45)
ax.legend()

# Add value labels
for i in range(len(existing_segments)):
    total = loyal_vals.iloc[i] + new_vals.iloc[i]
    ax.text(i, total + 0.5, str(int(total)), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('customer_type_by_segment.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: customer_type_by_segment.png")

## 7. Export Results

In [ ]:
# Prepare final results dataframe
rfm_results = rfm[['user_id', 'user_type', 'recency_days', 'frequency', 'monetary',
                   'r_score', 'f_score', 'm_score', 'rfm_score', 'segment', 'segment_description']].copy()

# Sort by segment and monetary value
segment_sort_order = {'Champion': 1, 'Loyalist': 2, 'Emerging': 3, 'Newbie': 4, 'At Risk': 5, 'Hibernating': 6}
rfm_results['segment_sort'] = rfm_results['segment'].map(segment_sort_order)
rfm_results = rfm_results.sort_values(['segment_sort', 'monetary'], ascending=[True, False])
rfm_results = rfm_results.drop('segment_sort', axis=1)

# Save to CSV
rfm_results.to_csv('rfm_results.csv', index=False)
print(f"Saved: rfm_results.csv ({rfm_results.shape[0]} customers)")
rfm_results.head(10)

In [ ]:
# Create summary statistics
summary_data = []

for segment in segment_order:
    seg_df = rfm[rfm['segment'] == segment]
    
    if len(seg_df) > 0:
        loyal_count = seg_df[seg_df['user_type'] == 'loyal'].shape[0]
        new_count = seg_df[seg_df['user_type'] == 'new'].shape[0]
        
        summary_data.append({
            'segment': segment,
            'description': segment_descriptions[segment],
            'customer_count': len(seg_df),
            'percentage': round(len(seg_df) / len(rfm) * 100, 2),
            'loyal_customers': loyal_count,
            'new_customers': new_count,
            'total_revenue': round(seg_df['monetary'].sum(), 2),
            'revenue_share_%': round(seg_df['monetary'].sum() / rfm['monetary'].sum() * 100, 2),
            'avg_recency_days': round(seg_df['recency_days'].mean(), 2),
            'avg_frequency': round(seg_df['frequency'].mean(), 2),
            'avg_monetary': round(seg_df['monetary'].mean(), 2),
            'avg_r_score': round(seg_df['r_score'].mean(), 2),
            'avg_f_score': round(seg_df['f_score'].mean(), 2),
            'avg_m_score': round(seg_df['m_score'].mean(), 2)
        })

rfm_summary = pd.DataFrame(summary_data)
rfm_summary.to_csv('rfm_summary.csv', index=False)
print(f"Saved: rfm_summary.csv")
rfm_summary

## 8. Business Insights & Recommendations

In [ ]:
print("="*80)
print("                    RFM SEGMENTATION INSIGHTS & RECOMMENDATIONS")
print("="*80)

total_customers = len(rfm)
total_revenue = rfm['monetary'].sum()

print(f"\n{'OVERALL STATISTICS':^80}")
print("-"*80)
print(f"Total Customers Analyzed: {total_customers}")
print(f"Total Revenue: ${total_revenue:,.2f}")
print(f"Average Revenue per Customer: ${total_revenue/total_customers:,.2f}")

for segment in segment_order:
    seg_df = rfm[rfm['segment'] == segment]
    if len(seg_df) > 0:
        count = len(seg_df)
        pct = count / total_customers * 100
        rev = seg_df['monetary'].sum()
        rev_pct = rev / total_revenue * 100
        avg_freq = seg_df['frequency'].mean()
        avg_rec = seg_df['recency_days'].mean()
        
        print(f"\n{'='*80}")
        print(f"SEGMENT: {segment.upper()}")
        print(f"{'='*80}")
        print(f"Description: {segment_descriptions[segment]}")
        print(f"\nKey Metrics:")
        print(f"  - Customers: {count} ({pct:.1f}% of total)")
        print(f"  - Revenue: ${rev:,.2f} ({rev_pct:.1f}% of total)")
        print(f"  - Avg Transactions: {avg_freq:.1f}")
        print(f"  - Avg Days Since Last Purchase: {avg_rec:.0f}")
        
        # Segment-specific recommendations
        if segment == 'Champion':
            print(f"\nRecommendations:")
            print(f"  - Reward with exclusive VIP benefits and early access to new products")
            print(f"  - Encourage referrals with ambassador programs")
            print(f"  - Seek feedback for product improvements")
        elif segment == 'Loyalist':
            print(f"\nRecommendations:")
            print(f"  - Upsell premium products and bundles")
            print(f"  - Offer loyalty rewards and points programs")
            print(f"  - Send personalized product recommendations")
        elif segment == 'Emerging':
            print(f"\nRecommendations:")
            print(f"  - Nurture with targeted promotions based on purchase history")
            print(f"  - Encourage more frequent visits with time-limited offers")
            print(f"  - Cross-sell complementary products")
        elif segment == 'Newbie':
            print(f"\nRecommendations:")
            print(f"  - Welcome email series with product highlights")
            print(f"  - First-time buyer discounts on next purchase")
            print(f"  - Educational content about product benefits")
        elif segment == 'At Risk':
            print(f"\nRecommendations:")
            print(f"  - URGENT: Send re-engagement campaigns immediately")
            print(f"  - Offer 'We miss you' discounts")
            print(f"  - Survey to understand why they stopped visiting")
        elif segment == 'Hibernating':
            print(f"\nRecommendations:")
            print(f"  - Win-back campaigns with significant discounts")
            print(f"  - Highlight new products since their last visit")
            print(f"  - Consider if re-acquisition cost is worth it")

print(f"\n{'='*80}")
print("                              END OF ANALYSIS")
print(f"{'='*80}")

In [ ]:
# Final verification
print("\n" + "="*60)
print("VERIFICATION SUMMARY")
print("="*60)
print(f"\nTotal customers segmented: {len(rfm_results)}")
print(f"\nSegment breakdown:")
for segment in segment_order:
    count = len(rfm_results[rfm_results['segment'] == segment])
    print(f"  - {segment}: {count}")

print(f"\nOutput files generated:")
print(f"  - rfm_results.csv")
print(f"  - rfm_summary.csv")
print(f"  - rfm_distributions.png")
print(f"  - segment_pie_chart.png")
print(f"  - segment_bar_charts.png")
print(f"  - frequency_vs_monetary_scatter.png")
print(f"  - rfm_heatmap.png")
print(f"  - customer_type_by_segment.png")

print(f"\nAnalysis complete!")